In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
from keras.layers import Activation, concatenate, Conv1D, Dense, Dropout, Flatten, Input, Lambda
from keras.models import Model
from generator import AudioGenerator, kltls, labels_to_ys, ys_to_labels
import pickle
import numpy as np
import datetime
import math
from sklearn.metrics import hamming_loss
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt

# Allows me to import my modules
sys.path.append('./modules')
from audio_utils import *

Using TensorFlow backend.


Attempting to read settings file...
	Read successfully!


In [2]:
3//2

1

In [2]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16700123651963006759, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 577778483
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1395718361558479677
 physical_device_desc: "device: 0, name: GeForce GTX 650, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [3]:
# Tells Tensorflow to use the GPU
config = tf.ConfigProto(allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 0},
                        log_device_placement = True
                       )

session = tf.Session(config=config)
K.set_session(session)
model_name = "modelA-bce-adam_" + str(datetime.datetime.now().strftime('%d-%m-%Y_%H-%M-%S'))

In [4]:
# Data generators
batch_size = 40
generators = {"training": None, "validation": None, "test": None}
N = {"training": 0, "validation": 0, "test": 0}
for data_type in generators.keys():
    sample_metadata = get_file_classes(data_type)
    print(sample_metadata[0:2])
    N[data_type] = len(sample_metadata)
    filenames = [sm["filepath"] for sm in sample_metadata]
    labels = [sm["labels"] for sm in sample_metadata]
    generators[data_type] = AudioGenerator(filenames, labels, data_type, batch_size, shuffle=True)

[{'filepath': 'audio_data\\training_data\\beater\\bass_drum\\normal\\0.gz', 'labels': {'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}, 'augmentations': {}}, {'filepath': 'audio_data\\training_data\\beater\\bass_drum\\normal\\1.gz', 'labels': {'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}, 'augmentations': {}}]
[{'filepath': 'audio_data\\validation_data\\beater\\bass_drum\\normal\\0.gz', 'labels': {'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}, 'augmentations': {}}, {'filepath': 'audio_data\\validation_data\\beater\\bass_drum\\normal\\1.gz', 'labels': {'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}, 'augmentations': {}}]
[{'filepath': 'audio_data\\test_data\\beater\\bass_drum\\normal\\0.gz', 'labels': {'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}, 'augmentations': {}}, {'filepath': 'audio_data\\test_data\\beater\\bass_drum\\n

In [5]:
generators["training"].on_epoch_end()
batch_0 = generators["training"].__getitem__(0)
for y in batch_0[1]:
    print(ys_to_labels(y))

{'hit_label': ['stick'], 'kit_label': ['ride'], 'tech_label': ['normal']}
{'hit_label': ['beater', 'stick', 'stick'], 'kit_label': ['bass_drum', 'low_tom', 'mid_tom'], 'tech_label': ['normal', 'normal', 'normal']}
{'hit_label': ['stick', 'stick'], 'kit_label': ['hi_hat', 'crash'], 'tech_label': ['normal', 'normal']}
{'hit_label': ['beater', 'stick'], 'kit_label': ['bass_drum', 'hi_hat'], 'tech_label': ['normal', 'normal']}
{'hit_label': ['beater', 'stick', 'stick'], 'kit_label': ['bass_drum', 'hi_hat', 'snare'], 'tech_label': ['normal', 'normal', 'normal']}
{'hit_label': ['beater', 'stick', 'stick'], 'kit_label': ['bass_drum', 'high_tom', 'snare'], 'tech_label': ['normal', 'normal', 'normal']}
{'hit_label': ['beater', 'stick', 'stick'], 'kit_label': ['bass_drum', 'hi_hat', 'low_tom'], 'tech_label': ['normal', 'normal', 'normal']}
{'hit_label': ['stick', 'stick'], 'kit_label': ['hi_hat', 'ride'], 'tech_label': ['open', 'bell']}
{'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech

In [6]:
batch_y_shape = None 
for name, gen in generators.items():
    print("Gen", name)
    batch_0 = gen.__getitem__(0)
    print("In shape:", batch_0[0].shape, "\nOut shape:", batch_0[1].shape)
    batch_y_shape = batch_0[1].shape

Gen training
In shape: (40, 12000, 1) 
Out shape: (40, 10)
Gen validation
In shape: (40, 12000, 1) 
Out shape: (40, 10)
Gen test
In shape: (40, 12000, 1) 
Out shape: (40, 10)


In [7]:
# Test whether generator arguments are picklable (whether they can be multiprocessed)
use_multiprocessing = True
for gen in generators:
    try:
        pickle.dumps(gen)
    except:
        print(sys.exc_info())
        use_multiprocessing = False
        break
print("Picklable:", use_multiprocessing)

Picklable: True


In [8]:
# Adapted from https://keras.io/layers/writing-your-own-keras-layers/
def InceptionModule(model):
    # Skip connection (uses input in concat)
    skip = Lambda(lambda x: x)(model)
    # Size 1 kernel conv of input (with tanh activation)
    conv_1_tower = Conv1D(filters=32, kernel_size=1, strides=1, padding="valid", kernel_initializer='glorot_normal', activation="tanh")(model)
    # Size 1 -> size 3 kernel conv of input (with tanh activation)
    conv_3_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_3_tower = Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", kernel_initializer='glorot_normal', activation="tanh")(conv_3_tower)
    # Size 1 -> size 5 kernel conv of input (with tanh activation)
    conv_5_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_5_tower = Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", kernel_initializer='glorot_normal', activation="tanh")(conv_5_tower)
    # Size 1 -> size 7 kernel conv of input (with tanh activation)
    conv_7_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_7_tower = Conv1D(filters=32, kernel_size=7, strides=1, padding="causal", kernel_initializer='glorot_normal', activation="tanh")(conv_7_tower)
    # Concatenate all activation images
    return concatenate([skip, conv_1_tower, conv_3_tower, conv_5_tower, conv_7_tower], axis=2)

In [9]:
# Reusable dilated convolution / inception module / dropout layer
def DilatedInceptionModuleLayer(model, drop_rate):
    model = Conv1D(filters=32, kernel_size=1, padding="causal", dilation_rate=2, kernel_initializer='glorot_normal', activation="tanh")(model)
    model = InceptionModule(model)
    return Dropout(rate=drop_rate)(model)

dim_rates = [0.1, 0.15, 0.2, 0.25, 0.3]

# Structure
"""
Rationale: 

3 "CausalConvAct" convolution layers which reduce the size of the sample space while increasing the size of the convolution space.
- Providing downscaling
(Feature extraction, while preserving temporal relationships)

Then "DilatedInceptionModule" layers which retain the size of the sample space while extracting more features.

- Using Convolutions to downsample from LeNet (?)
- Dropout paper
- ResNet for skip connections
- Inception module adapted from GoogLeNet
- Causal convolutions from WaveNet
"""
data = Input(shape=(12000, 1))
cnn = Conv1D(filters=16, kernel_size=7, strides=3, padding="causal", dilation_rate=1, kernel_initializer='glorot_normal', activation="tanh")(data)
cnn = Conv1D(filters=32, kernel_size=7, strides=2, padding="causal", dilation_rate=1, kernel_initializer='glorot_normal', activation="tanh")(cnn)
cnn = Conv1D(filters=32, kernel_size=5, strides=2, padding="causal", dilation_rate=1, kernel_initializer='glorot_normal', activation="tanh")(cnn)
cnn = Dropout(rate=0.1)(cnn)
for drop_rate in dim_rates:
    cnn = DilatedInceptionModuleLayer(cnn, drop_rate)
cnn = Flatten()(cnn)
cnn = Dense(10, kernel_initializer='glorot_normal', activation='sigmoid')(cnn)
model = Model(inputs=data, outputs=cnn)

for layer in model.layers:
    print(layer.name, layer.output_shape)

# Tensorboard logs
tb_logs = TensorBoard(log_dir="logs/{}".format(model_name))

def hamming_loss(threshold, output_shape):
    def hamming(y_true, y_pred):
        thr = tf.fill(value=threshold, dims=output_shape)
        y_pred_thresholded = K.cast(K.greater_equal(y_pred, thr), dtype=y_true.dtype)
        dist_tensor = K.cast(K.not_equal(y_true, y_pred_thresholded), dtype=y_true.dtype)
        dist = K.sum(dist_tensor)
        return 1/(output_shape[0]*output_shape[1])*dist
    return hamming

# Compile with stocastic gradient descent and mean squared error loss (same as multilabelled paper)
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy", hamming_loss(0.7, batch_y_shape)])#, options=run_opts)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
input_1 (None, 12000, 1)
conv1d_1 (None, 4000, 16)
conv1d_2 (None, 2000, 32)
conv1d_3 (None, 1000, 32)
dropout_1 (None, 1000, 32)
conv1d_4 (None, 1000, 32)
conv1d_6 (None, 1000, 1)
conv1d_8 (None, 1000, 1)
conv1d_10 (None, 1000, 1)
lambda_1 (None, 1000, 32)
conv1d_5 (None, 1000, 32)
conv1d_7 (None, 1000, 32)
conv1d_9 (None, 1000, 32)
conv1d_11 (None, 1000, 32)
concatenate_1 (None, 1000, 160)
dropout_2 (None, 1000, 160)
conv1d_12 (None, 1000, 32)
conv1d_14 (None, 1000, 1)
conv1d_16 (None, 1000, 1)
conv1d_18 (None, 1000, 1)
lambda_2 (None, 1000, 32)
conv1d_13 (None, 1000, 32)
conv1d_15 (None, 1000, 32)
conv1d_17 (None, 1000, 32)
conv1d_19 (None, 1000, 32)
concatenate_2 (None, 1000, 160)
dropout_3 (None, 1000, 160)
conv1d_20 (None, 1000, 32)
conv1d_22 (None, 1000, 1)
conv1d_24 (None, 1000, 1)
conv1d_26 (N

In [10]:
# Train model
epochs = 5
dataset_perc = 1
training_history = model.fit_generator(
                generator = generators["training"],
                steps_per_epoch = int((N["training"]*dataset_perc) // batch_size),
                validation_data = generators["validation"],
                validation_steps = int((N["validation"]*dataset_perc) / batch_size),
                epochs = epochs,
                callbacks = [tb_logs]
                #use_multiprocessing = use_multiprocessing,
                #workers = 4
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
2410/2410 [==============================] - 8776s 4s/step - loss: 0.4388 - acc: 0.8322 - hamming: 0.1719 - val_loss: 0.4024 - val_acc: 0.8468 - val_hamming: 0.1572
Epoch 2/5
2410/2410 [==============================] - 6913s 3s/step - loss: 0.3930 - acc: 0.8499 - hamming: 0.1548 - val_loss: 0.3669 - val_acc: 0.8573 - val_hamming: 0.1484
Epoch 3/5
2410/2410 [==============================] - 6974s 3s/step - loss: 0.3644 - acc: 0.8585 - hamming: 0.1466 - val_loss: 0.3440 - val_acc: 0.8651 - val_hamming: 0.1423
Epoch 4/5
2410/2410 [==============================] - 6974s 3s/step - loss: 0.3458 - acc: 0.8655 - hamming: 0.1418 - val_loss: 0.3277 - val_acc: 0.8719 - val_hamming: 0.1378
Epoch 5/5
2410/2410 [==============================] - 7232s 3s/step - loss: 0.3309 - acc: 0.8714 - hamming: 0.1371 - val_loss: 0.3128 - val_acc: 0.8792 - val_hamming: 0.1323


In [ ]:
# Evaluate model
model.evaluate_generator(
    generators["test"],
    int((N["test"]*dataset_perc) // batch_size)
    #use_multiprocessing = use_multiprocessing,
    #workers = 4
)

In [ ]:
save_dir = os.path.join(os.getcwd(), "models")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model_json = model.to_json()
with open(os.path.join(save_dir, "{}.json".format(model_name)), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(save_dir, "{}.h5".format(model_name)))
print("Saved model to disk")

In [ ]:
generators["test"].on_epoch_end()
batch0_test = generators["test"].__getitem__(0)

In [ ]:
predictions=[]
for i in range(batch_size):
    x, y = batch0_test[0][i], batch0_test[1][i]
    pred_y = np.reshape(model.predict(x.reshape(1, 12000, 1)), 10)
    print(pred_y.shape)
    print("Actual:\n\t{},\n\t{}\nPrediction:\n\t{},\n\t{}\n".format(y, ys_to_labels(y), [round(p_y, 3) for p_y in pred_y.tolist()], ys_to_labels([int(round(p_y)) for p_y in pred_y.tolist()])))
    predictions.append({"pred": pred_y, "actual": y})
predictions